# LSTM for Spoken Digit Classification

This notebook builds and trains a recurrent neural network (LSTM) to classify spoken digits (0–9) from audio recordings.

- Dataset: [Free Spoken Digit Dataset (FSDD)](https://github.com/Jakobovski/free-spoken-digit-dataset)
- Framework: PyTorch
- Architecture: RNN with LSTM layers

In [1]:
import sys
import os

# Add the project root (parent of current folder) to Python path
project_root_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(project_root_dir)


## Load Model Configuration from YAML

To make the training pipeline configurable and modular, we store model parameters like number of LSTM layers, hidden size, and learning rate etc in a YAML file. This structure enables quick adaptation to related tasks B, and C.

This section loads the model configuration using a custom utility function.

In [2]:
import src.utils as utils

In [4]:
import yaml
import json

model_config_path = os.path.join(project_root_dir, 'config', 'model_config.yaml')
model_config = utils.read_yaml_file(model_config_path)
# print(json.dumps(model_config, indent=2))

## Load and Split Dataset for Training and Evaluation

In this section, we load the recordings data from disk, generate data-label pairs, and split them into training and test sets according to the `test_size` defined in the YAML file.

Using `test_size` and `seed` from the YAML config ensures that experiments are reproducible and easily tunable for other tasks by simply updating the configuration.


In [4]:
data_path = model_config['dataset']['path']
test_data_size = model_config['data_splitting']['test_size']
seed = model_config['experiment']['seed']

In [5]:
data_label_pairs = utils.prepare_data_label_pairs(data_path)

In [6]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(data_label_pairs, test_size=test_data_size, random_state=seed)

## Transform Raw Data into PyTorch Dataset Objects

The `AudioFeaturesDataset` class converts raw data-label pairs into PyTorch-compatible datasets that provide easy access to samples and labels.

AudioFeaturesDataset is a custom dataset class that:

- Loads audio recordings of spoken digits along with their labels.
- Optionally cleans the audio by filtering out noise.
- Extracts MFCC features (a common speech feature).
- Pads or trims these features to a fixed length so all inputs have the same shape.
- Works with PyTorch to provide samples one-by-one when training or testing a model.
- It helps prepare your audio data in the right format for training neural networks efficiently.


In [7]:
from src.data_preprocessor import AudioFeaturesDataset

train_dataset = AudioFeaturesDataset(train_data)
test_dataset = AudioFeaturesDataset(test_data)

In [9]:
print(f"Train size: {len(train_dataset)}")
print(f"Test size: {len(test_dataset)}")

Train size: 2400
Test size: 600


## Create DataLoaders for Batch Processing

Using PyTorch DataLoaders, we enable efficient loading, batching, and shuffling of data during training and evaluation.

In [10]:
import torch

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

## LSTM Model Definition

A simple `n`-layer LSTM followed by a fully connected output layer. Variable `n` is defined in the configuration YAML file

In [11]:
input_dim = model_config['model']['input_dim']
hidden_dim = model_config['model']['hidden_dim']
num_layers = model_config['model']['num_layers']
output_dim = model_config['model']['output_dim']

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [13]:
import src.model as model
import torch.nn as nn
import torch.optim as optim

model = model.LSTMClassifier(input_dim=input_dim,
                       hidden_dim=hidden_dim,
                       num_layers=num_layers,
                       output_dim=output_dim).to(device)

## Training Loop

In [14]:
learning_rate = model_config['training']['learning_rate']
epochs = model_config['training']['epochs']

In [15]:
from src.train import ModelTrainer
trainer_instance = ModelTrainer(
    model, 
    epochs,
    train_loader,
    device,
    learning_rate
)

Training INitialised


In [16]:
trainer_instance.train()

Epoch [1/5], Loss: 139.4414, Accuracy: 64.88%
Epoch [2/5], Loss: 49.1270, Accuracy: 89.00%
Epoch [3/5], Loss: 29.4020, Accuracy: 90.46%
Epoch [4/5], Loss: 12.9803, Accuracy: 98.04%
Epoch [5/5], Loss: 8.9003, Accuracy: 96.00%


In [17]:
utils.get_model_params_size(model)


Layer-wise parameter counts:
lstm.weight_ih_l0              -> 6,656 params
lstm.weight_hh_l0              -> 65,536 params
lstm.bias_ih_l0                -> 512 params
lstm.bias_hh_l0                -> 512 params
lstm.weight_ih_l1              -> 65,536 params
lstm.weight_hh_l1              -> 65,536 params
lstm.bias_ih_l1                -> 512 params
lstm.bias_hh_l1                -> 512 params
fc.weight                      -> 1,280 params
fc.bias                        -> 10 params


 Total Parameters: 206,602
Estimated Memory: 807.04 KB (0.79 MB)


({'lstm.weight_ih_l0': 6656,
  'lstm.weight_hh_l0': 65536,
  'lstm.bias_ih_l0': 512,
  'lstm.bias_hh_l0': 512,
  'lstm.weight_ih_l1': 65536,
  'lstm.weight_hh_l1': 65536,
  'lstm.bias_ih_l1': 512,
  'lstm.bias_hh_l1': 512,
  'fc.weight': 1280,
  'fc.bias': 10},
 807.0390625)

## Evaluation & Visualization

In [18]:
from src.evaluate import ModelEvaluator

In [19]:
test_instance = ModelEvaluator(
    model, 
    test_loader,
    device
)


In [21]:
test_instance.evaluate()

Inside here
Accuracy on test data: 92.17%
